In [1]:
%config Completer.use_jedi = False

## Bayesian Probability

#### Probability of group A's CVR is higher that of group B's  
: $P(A>B)$

#### Probability of group B's CVR is higher than that of group A's  
: $P(B>A)$

$a : \text{Conversion rate of A}$  
$b : \text{Conversion rate of B}$ 

$$P(A>B)=\int_{a=0}^{1}\int_{b=0}^{a} P(a|A) * P(b|B)$$

### 간단한 테스트 상황에서는 잘 동작하지만 실전에서 사용할 때 두 가지 문제가 있다. 
- 첫째, 전환율을 계산할 때 시행 횟수와 전환 횟수가 매우 큰 경우에는 "함수가 정확한 적분값을 계산하지 못한다. 이것은 정밀도 문제 때문에 생기는 현상인데 가령 아래와 같은 경우는 엉뚱한 값을 출력한다. 
 
 
- 또 다른 문제점은 정량값 확률을 계산하는  만약 확률을 계산할 그룹의 정량값의 분포가 정규분포와 크게 다르다면(예를 들어 멱함수 분포) 확률이 부정확하게 계산될 수 있다. 이 문제를 해결하려면 해당 그룹의 데이터 분포에 맞게 dnorm과 pnorm 함수 호출 부분을 수정해줘야 한다. 특히 만약 데이터의 분포가 모수적 방법으로 표현할 수 없는 분포라면 비모수적 기법(non-parametric method)을 이용해 확률 분포를 계산해 줘야 한다.

### In R code
- 접두사	의미
    - d : 확률 밀도(질량) 함수 (probability density/mass function)
    - p : 누적 분포 함수 (cumulative distribution function)
    - q : 분위수 계산 함수 (quantile function)
    - r : 랜덤 샘플 생성 (sample realization)

``` R
bayesian.stat.for.rate<-function(data){
  k <- nrow(data)
  result<-numeric(k)
  for(i in (1:k)){
    idx<-(1:k)[-i]
    f<-function(z){
      r<-dbeta(z,
               data[i, 'conversion']+1,
               data[i, 'trial'] - data[i, 'conversion'] + 1)
      for (j in idx){
        r<-r*pbeta(z,
                   data[j, 'conversion'] + 1,
                   data[j, 'trial'] - data[j, 'conversion'] + 1)
      }
      return(r)
    }
    result[i] = integrate(f, 0,1)$value 
    }
  return(result)
}
``` 

In [45]:
seta

{1, 2, 3, 4}

In [48]:
a = [1,2,3,4]
seta = set(a)

for i in range(1,5,1):
    print(seta-{i})

{2, 3, 4}
{1, 3, 4}
{1, 2, 4}
{1, 2, 3}


In [47]:
a = [1,2,3,4]
seta = set(a)

for i in a:
    print(seta-{i})

{2, 3, 4}
{1, 3, 4}
{1, 2, 4}
{1, 2, 3}


In [56]:
import pandas as pd

In [59]:
data = {'group' : ['A','B','C'],
       'trial' : [8500, 8500, 8500],
       'conversion' : [1500, 1410, 1420]}

In [62]:
df_data = pd.DataFrame(data)

In [54]:
def bayesian_stat_for_rate(data):
    k = len(data)
    for i in range(1,k+1):
        lst = list(range(1,k+1))
        set_lst = set(lst)
        index = set_lst-{i}
        f = lambda(x : beta.pdf(data.iloc[i, 2]+1, beta.pdf(data.iloc[i,1] - data.iloc[i,2]+1)))
        for j in range(index):
            r = lambda(y : beta.cdf(data.iloc[j, 2]+1, beta.cdf(data.ilco[j,1] - data.iloc[j,2]+1)))
            
        result[i] = integrate(f,0,1)
        
    return result

In [55]:
bayesian_stat_for_rate([1,2,3,4,5])

{2, 3, 4, 5}
{1, 3, 4, 5}
{1, 2, 4, 5}
{1, 2, 3, 5}
{1, 2, 3, 4}


In [ ]:
dbeta = beta.pdf()
pbeta = beta.cdf()
integrate.quad(f, 0,1)

In [1]:
from scipy import integrate
from scipy.stats import beta
import numpy as np

In [ ]:
# dbeta ==> beta.pdf (density function)
# pbeta ==> beta.rvs
# dbeta gives the density, pbeta the distribution function, 

In [23]:
conversion_a = 3300
conversion_b = 170

total_a = 10000
total_b = 500

prob_A_better_B = np.multiply(beta.rvs(1+conversion_a, 1+total_a-conversion_a, size=1),\
                              beta.rvs(1+conversion_b, 1+total_b-conversion_b))

prob_A_better_B

# f= lambda x:exp(-x**2)
# i = scipy.integrate.quad(f, 0, 1)

array([0.12231251])

In [26]:
# f = np.multiply(beta.rvs(1+conversion_a, 1+total_a-conversion_a, size=1),\
#                               beta.rvs(1+conversion_b, 1+total_b-conversion_b))
# i = integrate.quad(f, 0, 1)
# result = integrate.quad(lambda x: special.jv(2.5,x), 0, 4.5)

integrate.quad(lambda x,y : beta.pdf(1+conversion_a, 1+total_a-conversion_a, size=1),\
                              beta.pdf(1+conversion_b, 1+total_b-conversion_b), 0,1)

TypeError: _parse_args() missing 1 required positional argument: 'b'

In [16]:
f= lambda x:np.exp(-x**2)
i = integrate.quad(f, 0, 1)

In [19]:
i

(0.7468241328124271, 8.291413475940725e-15)